In [1]:
pip install tensorflow python-dotenv pandas numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


## Import Necessary Libraries

In [2]:
import os
import requests
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv

## Load Data

In [8]:
load_dotenv()  # Load environment variables from .env file
jwt_token = os.getenv('JWT_TOKEN')

headers = {'Authorization': f'Bearer {jwt_token}'}

api_urls = {
    'interactions': 'http://161.97.109.65:3000/api/interactions',
    'users': 'http://161.97.109.65:3000/api/users',
    'products': 'http://161.97.109.65:3000/api/products'
}

def fetch_data(url, headers):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raises an HTTPError for bad requests
        data = pd.DataFrame(response.json())
        print(f"Data successfully fetched from {url}")
        print(data.head())  # Display the first few rows of the DataFrame
        return data
    except requests.RequestException as e:
        print(f'Failed to fetch data from {url}: {str(e)}')
        return pd.DataFrame()

# Fetch data from APIs
users = fetch_data(api_urls['users'], headers)
products = fetch_data(api_urls['products'], headers)
interactions = fetch_data(api_urls['interactions'], headers)

# Check if data was fetched successfully
if not users.empty and not products.empty and not interactions.empty:
    print("All data fetched successfully.")
else:
    print("Data fetching failed, check errors and retry.")
    # Optionally, add logic to halt further processing if data is crucial


Data successfully fetched from http://161.97.109.65:3000/api/users
                        _id       firstname       lastname         username  \
0  6665e9847aa0dfec0ad43b26         Machine       Learning  machinelearning   
1  6665eab57aa0dfec0ad43b2a  DummyFirstname  DummyLastname       dummydata1   
2  6665eac87aa0dfec0ad43b2d  DummyFirstname  DummyLastname       dummydata2   
3  6665eacc7aa0dfec0ad43b30  DummyFirstname  DummyLastname       dummydata3   
4  6665eacf7aa0dfec0ad43b33  DummyFirstname  DummyLastname       dummydata4   

                  email         phone  \
0          ml@admin.com    7777777777   
1  dummydata1@admin.com  777777770001   
2  dummydata2@admin.com  777777770002   
3  dummydata3@admin.com  777777770003   
4  dummydata4@admin.com  777777770004   

                                            password           address  \
0  $2a$10$oNSoSQcmxvHAefk5dKx0UuJw8oSdGeCumA.ZqIN...  Bangkit Capstone   
1  $2a$10$ihldsbescWBR9v94/sRhReBpX8mZMGrRpwkUohU...   Dummy Ad

## Data Preprocessing

In [4]:
# Let's assume 'interactions' is a DataFrame with a column containing dictionaries
# First, ensure that the 'interactions' column is appropriately normalized
if 'interactions' in interactions.columns:
    interactions_expanded = pd.json_normalize(interactions['interactions'])
else:
    interactions_expanded = pd.json_normalize(interactions.iloc[:, 0])  # If 'interactions' is the name of DataFrame and not a column

# Assuming the JSON data has keys 'userId', 'productId', and 'interactionValue'
interactions_expanded['user_id'] = interactions_expanded['userId']
interactions_expanded['product_id'] = interactions_expanded['productId']
interactions_expanded['interaction_value'] = interactions_expanded['interactionValue']

# Encode user_id and product_id
user_ids = interactions_expanded['user_id'].unique().tolist()
product_ids = interactions_expanded['product_id'].unique().tolist()

user2user_encoded = {x: i for i, x in enumerate(user_ids)}
product2product_encoded = {x: i for i, x in enumerate(product_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
productencoded2product = {i: x for i, x in enumerate(product_ids)}

interactions_expanded['user'] = interactions_expanded['user_id'].map(user2user_encoded)
interactions_expanded['product'] = interactions_expanded['product_id'].map(product2product_encoded)

# Split the data
train, test = train_test_split(interactions_expanded, test_size=0.2, random_state=42)

# Convert data into required format
x_train = [train['user'].values, train['product'].values]
y_train = train['interaction_value'].values
x_test = [test['user'].values, test['product'].values]
y_test = test['interaction_value'].values

## Define Model

In [5]:
user_input = Input(shape=(1,))
user_embedding = Embedding(len(user2user_encoded), 50)(user_input)  # Removed input_length
user_vec = Flatten()(user_embedding)

product_input = Input(shape=(1,))
product_embedding = Embedding(len(product2product_encoded), 50)(product_input)  # Removed input_length
product_vec = Flatten()(product_embedding)

dot_product = Dot(axes=1)([user_vec, product_vec])
model = Model(inputs=[user_input, product_input], outputs=dot_product)
model.compile(optimizer=Adam(), loss='mean_squared_error')


## Train Model

In [6]:
# Assuming 'train' and 'test' datasets are already split and preprocessed
x_train = [np.array(train['user']), np.array(train['product'])]
y_train = np.array(train['interaction_value'])

# Fit the model
history = model.fit(
    x_train, 
    y_train, 
    epochs=20, 
    verbose=1, 
    validation_split=0.2
)

# Optionally, you can add callbacks, for example to save the best model or early stopping
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Setup callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
]

# Fit the model with callbacks
history = model.fit(
    x_train, 
    y_train, 
    epochs=20, 
    verbose=1, 
    validation_split=0.2,
    callbacks=callbacks
)

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 1.0022 - val_loss: 10.0033
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.9973 - val_loss: 10.0033
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.9924 - val_loss: 10.0033
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.9875 - val_loss: 10.0033
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.9826 - val_loss: 10.0033
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.9777 - val_loss: 10.0033
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.9728 - val_loss: 10.0033
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.9678 - val_loss: 10.0033
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.9629 - val_loss: 10.0033
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.9579 - val_loss: 10.0033
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.9528 - val_loss: 10.0033
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.9477 - val_loss:

ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=best_model.h5

## Try Model

In [ ]:
# Function to get recommendations for a specific user
def recommend_products(user_id, model, interactions, user2user_encoded, product2product_encoded, productencoded2product, products, top_n=30):
    # Check if user_id is in the encoding map
    if user_id not in user2user_encoded:
        print(f"User ID {user_id} not found.")
        return pd.DataFrame()

    user_encoded = user2user_encoded[user_id]
    # Get all encoded product IDs as a list of integers
    product_ids = list(product2product_encoded.values())

    # Create user-product array for prediction
    # Ensure all entries are integers for the model input
    user_product_array = np.array([[user_encoded] * len(product_ids), product_ids]).T.astype(int)

    # Predict interaction values using the model
    predictions = model.predict([user_product_array[:, 0], user_product_array[:, 1]])
    predictions = predictions.flatten()

    # Get top N product indices
    top_indices = predictions.argsort()[-top_n:][::-1]
    # Decode the top indices to product IDs
    recommended_product_ids = [productencoded2product[x] for x in top_indices]

    # Filter the products DataFrame to get recommended products using the correct column name
    recommended_products = products[products['_id'].isin(recommended_product_ids)]
    return recommended_products

# Try the model with the specified user ID
user_id = '666545a42b9108ea2b463d87'
recommended_products = recommend_products(user_id, model, interactions, user2user_encoded, product2product_encoded, productencoded2product, products)
print(f"Recommended products for user {user_id}:")
print(recommended_products)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Recommended products for user 666545a42b9108ea2b463d87:
                        _id    name         category description  price  \
0  6665935c2b9108ea2b463dc2  bababa  asdadaasdadaasd  1234567890     25   

                   sellerId  \
0  666545a42b9108ea2b463d87   

                                        productImage  __v  
0  [https://storage.googleapis.com/kelas-app-test...    0  
